In [2]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import time
import numpy.linalg as ln
from scipy.sparse import csr_matrix

In [3]:
tree = ET.parse('padgett.xml')
root = tree.getroot()
print(tree.getroot())

<Element 'DynamicNetwork' at 0x00000145BDB4BE58>


In [4]:
for child in root:
        print(child.tag, child.attrib)
        if child.tag == "appointment":
            for step_child in child:
                print(step_child.tag)

MetaNetwork {}


In [5]:
print("-" * 40)
print("Iterating using a tree iterator")
print("-" * 40)
iter_ = tree.getiterator()
    
for elem in iter_:
    print(elem.tag)

----------------------------------------
Iterating using a tree iterator
----------------------------------------
DynamicNetwork
MetaNetwork
nodes
nodeclass
node
node
node
node
node
node
node
node
node
node
node
node
node
node
node
node
networks
network
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
link
l

In [6]:
for node in root.iter('node'):
     print(node.attrib)

{'id': 'ACCIAIUOL'}
{'id': 'ALBIZZI'}
{'id': 'BARBADORI'}
{'id': 'BISCHERI'}
{'id': 'CASTELLAN'}
{'id': 'GINORI'}
{'id': 'GUADAGNI'}
{'id': 'LAMBERTES'}
{'id': 'MEDICI'}
{'id': 'PAZZI'}
{'id': 'PERUZZI'}
{'id': 'PUCCI'}
{'id': 'RIDOLFI'}
{'id': 'SALVIATI'}
{'id': 'STROZZI'}
{'id': 'TORNABUON'}


In [7]:
# one specific item attribute 
print('Item #2 attribute:') 
print(root[0][0][0][0].attrib) 


# one specific item's data 
print('\nItem #2 data:') 
print(root[0][0][0][0].text) 

# all items data 
print('\nAll item data:') 
for elem in root: 
    for subelem in elem: 
        print(subelem.text)


Item #2 attribute:
{'id': 'ACCIAIUOL'}

Item #2 data:
None

All item data:
            
            


In [24]:
fileptr = open('ItalFam.txt', 'w')


In [22]:
def numb(x):
    if x == "ACCIAIUOL":
        return 1
    elif x == "ALBIZZI":
        return 2
    elif x == "BARBADORI":
        return 3
    elif x == "BISCHERI":
        return 4
    elif x == "CASTELLAN":
        return 5
    elif x == "GINORI":
        return 6
    elif x == "GUADAGNI":
        return 7
    elif x == "LAMBERTES":
        return 8
    elif x == "MEDICI":
        return 9
    elif x == "PAZZI":
        return 10
    elif x == "PERUZZI":
        return 11
    elif x == "PUCCI":
        return 12
    elif x == "RIDOLFI":
        return 13
    elif x == "SALVIATI":
        return 14
    elif x == "STROZZI":
        return 15
    elif x == "TORNABUON":
        return 16

In [26]:
for link in root.iter('link'):
    line=[str(numb(link.attrib['source'])),",",str(numb(link.attrib['target'])),"\n"]
    fileptr.writelines(line) 


In [27]:
fileptr.close()

In [11]:
nodes=[]

for node in root.iter('node'):
    nodes.append(node.attrib['id']) 

In [12]:
node_to_node=[]

for link in root.iter('link'):
    node_to_node.append((link.attrib['source'],link.attrib['target'])) 

In [13]:
import networkx as nx 

In [14]:
G = nx.DiGraph()

[G.add_node(k) for k in nodes]
G.add_edges_from(node_to_node)


ppr1 = nx.pagerank(G)

print("Page rank value: " + str(ppr1))

Page rank value: {'ACCIAIUOL': 0.062499999999999986, 'ALBIZZI': 0.062499999999999986, 'BARBADORI': 0.062499999999999986, 'BISCHERI': 0.062499999999999986, 'CASTELLAN': 0.062499999999999986, 'GINORI': 0.062499999999999986, 'GUADAGNI': 0.062499999999999986, 'LAMBERTES': 0.062499999999999986, 'MEDICI': 0.062499999999999986, 'PAZZI': 0.062499999999999986, 'PERUZZI': 0.062499999999999986, 'PUCCI': 0.062499999999999986, 'RIDOLFI': 0.062499999999999986, 'SALVIATI': 0.062499999999999986, 'STROZZI': 0.062499999999999986, 'TORNABUON': 0.062499999999999986}


In [15]:
class Node:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.parents = []
        self.auth = 1.0
        self.hub = 1.0
        self.pagerank = 1.0
        
    def link_child(self, new_child):
        for child in self.children:
            if(child.name == new_child.name):
                return None
        self.children.append(new_child)

    def link_parent(self, new_parent):
        for parent in self.parents:
            if(parent.name == new_parent.name):
                return None
        self.parents.append(new_parent)

    def update_auth(self):
        self.auth = sum(node.hub for node in self.parents)

    def update_hub(self):
        self.hub = sum(node.auth for node in self.children)

    def update_pagerank(self, d, n):
        in_neighbors = self.parents
        pagerank_sum = sum((node.pagerank / len(node.children)) for node in in_neighbors)
        random_jumping = d / n
        self.pagerank = random_jumping + (1-d) * pagerank_sum

In [16]:
class Graph:
    def __init__(self):
        self.nodes = []

    def contains(self, name):
        for node in self.nodes:
            if(node.name == name):
                return True
        return False

    def find(self, name):
        if(not self.contains(name)):
            new_node = Node(name)
            self.nodes.append(new_node)
            return new_node
        else:
            return next(node for node in self.nodes if node.name == name)

    def add_edge(self, parent, child):
        parent_node = self.find(parent)
        child_node = self.find(child)

        parent_node.link_child(child_node)
        child_node.link_parent(parent_node)

    def display(self):
        for node in self.nodes:
            print(f'{node.name} links to {[child.name for child in node.children]}')

    def sort_nodes(self):
        self.nodes.sort(key=lambda node: int(node.name))

    def display_hub_auth(self):
        for node in self.nodes:
            print(f'{node.name}  Auth: {node.old_auth}  Hub: {node.old_hub}')

    def normalize_auth_hub(self):
        auth_sum = sum(node.auth for node in self.nodes)
        hub_sum = sum(node.hub for node in self.nodes)

        for node in self.nodes:
            node.auth /= auth_sum
            node.hub /= hub_sum

    def normalize_pagerank(self):
        pagerank_sum = sum(node.pagerank for node in self.nodes)

        for node in self.nodes:
            node.pagerank /= pagerank_sum

    def get_auth_hub_list(self):
        auth_list = np.asarray([node.auth for node in self.nodes], dtype='float32')
        hub_list = np.asarray([node.hub for node in self.nodes], dtype='float32')

        return np.round(auth_list, 3), np.round(hub_list, 3)

    def get_pagerank_list(self):
        pagerank_list = np.asarray([node.pagerank for node in self.nodes], dtype='float32')
        return np.round(pagerank_list, 3)


In [18]:
def PageRank_one_iter(graph, d):
    node_list = graph.nodes
    for node in node_list:
        node.update_pagerank(d, len(graph.nodes))

In [19]:
def update_pagerank(self, d, n):
  in_neighbors = self.parents
  pagerank_sum = sum((node.pagerank / len(node.children)) for node in in_neighbors)
  random_walk = d / n
  self.pagerank = random_walk + (1-d) * pagerank_sum

In [29]:
def PageRank(graph, d, iteration=100):
    for i in range(int(iteration)):
        PageRank_one_iter(graph, d)


In [21]:



def init_graph(fname):
    with open(fname) as f:
        lines = f.readlines()

    graph = Graph()

    for line in lines:
        [parent, child] = line.strip().split(',')
        graph.add_edge(parent, child)

    graph.sort_nodes()

    return graph

In [33]:
iteration = 10000
damping_factor = 0.15

graph = init_graph('ItalFam.txt')

PageRank(iteration, graph, damping_factor)
print(graph.get_pagerank_list())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [34]:
mas = [0] * 16
for i in range(16): 
    mas[i] = [0] * 16 

In [62]:
for link in root.iter('link'):
    a=mas[numb(link.attrib['source'])-1][numb(link.attrib['target'])-1]
    a=a+1
    mas[numb(link.attrib['source'])-1][numb(link.attrib['target'])-1]=a

mas = np.array(mas, dtype=np.float32)
print(mas)

[[2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625
  2.0625 2.0625 2.0625 2.0625 2.0625 2.0625]
 [2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625 2.0625

In [61]:
summ = [0] * 16
for i in range(16):
    for j in range(16):
        summ[i]=summ[i]+mas[i][j]
        
for i in range(16):
    for j in range(16):
        if summ[i]==0:
            summ[i]==1
        mas[i][j]=mas[i][j]/summ[i]
        
print(mas)

[[0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625

In [54]:
def pagerank(M, d, num_iterations: int = 100):

    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    M_hat = (d * M + (1 - d) / N)
    for i in range(num_iterations):
        v = M_hat @ v
    return v


v = pagerank(mas, 0.85, 100)
print(v)

[[0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]
 [0.0625]]
